In [76]:
! pip install datasets transformers accelerate

In [77]:
!git clone https://github.com/koheisanno/dino_NLPFinal.git

fatal: destination path 'dino_NLPFinal' already exists and is not an empty directory.


In [78]:
import sys
from pathlib import Path

from dino_NLPFinal.utils import read_into_dict

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [79]:
from huggingface_hub import notebook_login

notebook_login()

Then you need to install Git-LFS. Uncomment the following instructions:

In [80]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [81]:
import transformers

print(transformers.__version__)

4.40.1


# Fine-tuning a model on a text classification task

In [82]:
task = "translation"
model_checkpoint = "FacebookAI/xlm-roberta-base"
batch_size = 16

## Loading the dataset

In [83]:
from datasets import load_dataset, load_metric, Dataset, ClassLabel
from importlib import reload
import pandas as pd

In [84]:
input_path = "dino_NLPFinal/crosslingual.pkl"
label_names = ["English", "French"]
# dataset = read_into_dict(input_path, "hypothesis", "premise")

df_raw = pd.read_pickle(input_path)
df = df_raw[df_raw['task'] == 'en;fr']

df = df.rename(columns={'stsb_test_1_lang1': 'en'})
df = df.rename(columns={'stsb_test_2_lang2': 'fr'})
df = df.rename(columns={'score_lang2': 'label'})
df = df.drop(columns=['task'])

def round_score(score):
    if score < 0.25:
        return 0.0
    elif score < 0.75:
        return 0.5
    else:
        return 1.0

df['label'] = df['label'].apply(round_score)

train_dataset = Dataset.from_pandas(df)
train_dataset = train_dataset.remove_columns(["__index_level_0__"])


train_dataset = train_dataset.train_test_split(test_size=0.2)
# Further splitting the test set into test and validation sets
test_val_dataset = train_dataset['test'].train_test_split(test_size=0.5)
val_dataset = test_val_dataset['test']
test_dataset = test_val_dataset['train']
train_dataset = train_dataset['train']

print((train_dataset[:10]))
print((val_dataset))
print((test_dataset))
# test_dataset, val_dataset = remainder.train_test_split(test_size=0.5)

# print(len(dataset['train']))
# print(len(train_dataset))
# dataset = Dataset.from_dict(dataset)
# dataset = dataset.cast_column("label", ClassLabel(names=label_names))

{'en': ['Electronic Data Systems Corp. Thursday said the Securities and Exchange Commission has asked the company for documents related to its large contract with the U.S. Navy.', 'A promotional poster, complete with countdown dial, reminds readers of the upcoming release of "Harry Potter and the Order of the Phoenix."', 'A man is dancing.', 'A panda bear is chewing on a stick.', 'Group of people posing with two men dressed as Elvis.', "State health officials said today a young northeast Kansas woman likely has the state's first case of monkeypox among humans or animals.", 'A chimp is kicking a target.', '123 killed in fighting in northern Syria', 'There are a few things you can do:', 'a young man wearing a white t-shirt and green and black shorts standing on a stump.'], 'fr': ["Dans un dossier réglementaire, EDS a déclaré que la SEC avait demandé des informations relatives à son contrat d'externalisation informatique en difficulté avec la marine américaine.", "Les caisses sont pleines

In [85]:
metric = load_metric('pearsonr')

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for pearsonr contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/pearsonr/pearsonr.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set (with more keys for the mismatched validation and test set in the special case of `mnli`).

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [86]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [106]:
show_random_elements(train_dataset)

,en,fr,label
0,You might have to try a variety before you find one that clicks with him.,Mon conseil serait d'essayer une variété de cafés que vous pouvez vous permettre et d'en trouver un que vous aimez.,0.5
1,A lady mixed up a meat mixture in bowl.,Une femme remue de la viande dans un bol.,1.0
2,"Yesterday, shares closed up 29 cents, or 0.54 percent, at $54.32.","Les actions d'Amazon ont clôturé hier à 54,32 $ sur le Nasdaq Stock Market, en hausse de 29 cents.",0.5
3,"IDEC shareholders would own 50.5 percent of the stock of the combined company, and Biogen shareholders the remaining 49.5 percent.","Les sociétés ont déclaré que les détenteurs d'Idec détiendraient environ 50,5% des actions de la société fusionnée.",0.5
4,The person is wearing a fedora.,Un homme porte un uniforme.,0.5
5,"Rich media doubled its share, increasing from 3% in Q2 2002 to 6% in Q2 2003.","Les formats d'annonces interactives Rich Media ont doublé leur part, passant de 3% au deuxième trimestre de 2002 à 6% au deuxième trimestre de 2003.",1.0
6,The flock of sheep are spread out in the field.,Un chat blanc se tient sur le sol.,0.0
7,The train sits at the train station.,Un train garé dans une gare.,1.0
8,A man is playing drums.,Un homme joue de sa guitare.,0.5
9,Georgian PM's ally to become president - exit poll,L'allié du Premier ministre milliardaire de Géorgie à la présidence des sondages,1.0


The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

In [107]:
metric

Metric(name: "pearsonr", features: {'predictions': Value(dtype='float32', id=None), 'references': Value(dtype='float32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted class labels, as returned by a model.
    references (`list` of `int`): Ground truth labels.
    return_pvalue (`boolean`): If `True`, returns the p-value, along with the correlation coefficient. If `False`, returns only the correlation coefficient. Defaults to `False`.

Returns:
    pearsonr (`float`): Pearson correlation coefficient. Minimum possible value is -1. Maximum possible value is 1. Values of 1 and -1 indicate exact linear positive and negative relationships, respectively. A value of 0 implies no correlation.
    p-value (`float`): P-value, which roughly indicates the probability of an The p-value roughly indicates the probability of an uncorrelated system producing datasets that have a Pearson correlation at least as extreme as the one computed from these datasets. Minimum possible v

You can call its `compute` method with your predictions and labels directly and it will return a dictionary with the metric(s) value:

In [108]:
import numpy as np

fake_preds = np.random.randint(0, 3, size=(64,))
fake_labels = np.random.randint(0, 3, size=(64,))
metric.compute(predictions=fake_preds, references=fake_labels)

{'pearsonr': 0.02672277115811787}

Note that `load_metric` has loaded the proper metric associated to your task, which is:

- for CoLA: [Matthews Correlation Coefficient](https://en.wikipedia.org/wiki/Matthews_correlation_coefficient)
- for MNLI (matched or mismatched): Accuracy
- for MRPC: Accuracy and [F1 score](https://en.wikipedia.org/wiki/F1_score)
- for QNLI: Accuracy
- for QQP: Accuracy and [F1 score](https://en.wikipedia.org/wiki/F1_score)
- for RTE: Accuracy
- for SST-2: Accuracy
- for STS-B: [Pearson Correlation Coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) and [Spearman's_Rank_Correlation_Coefficient](https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient)
- for WNLI: Accuracy

so the metric object only computes the one(s) needed for your task.

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [109]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


We pass along `use_fast=True` to the call above to use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [110]:
tokenizer("Hello, this one one one,,", "Hello this sentence goes with it.")

{'input_ids': [0, 35378, 4, 903, 1632, 1632, 1632, 4, 4, 2, 2, 35378, 903, 149357, 60899, 678, 442, 5, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

We can double check it does work on our current dataset:

In [111]:
sentence1_key, sentence2_key = ("en", "fr")
print(f"Sentence 1: {train_dataset[0][sentence1_key]}")
print(f"Sentence 2: {train_dataset[0][sentence2_key]}")

Sentence 1: Electronic Data Systems Corp. Thursday said the Securities and Exchange Commission has asked the company for documents related to its large contract with the U.S. Navy.
Sentence 2: Dans un dossier réglementaire, EDS a déclaré que la SEC avait demandé des informations relatives à son contrat d'externalisation informatique en difficulté avec la marine américaine.


We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model.

In [113]:
def preprocess_function(examples):
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [114]:
preprocess_function(train_dataset[:5])

{'input_ids': [[0, 87673, 11809, 84837, 89499, 5, 103616, 2804, 70, 503, 20287, 2449, 136, 104758, 63871, 1556, 37170, 70, 14380, 100, 60525, 62548, 47, 6863, 21334, 18264, 678, 70, 345, 5, 294, 5, 19295, 53, 5, 2, 2, 12612, 51, 60140, 35775, 19929, 9459, 4, 241, 13526, 10, 162235, 41, 21, 75018, 17095, 108406, 224, 32565, 35845, 7, 253, 775, 41061, 104, 25, 3355, 31586, 141, 15032, 165796, 22, 34844, 446, 1609, 21, 122084, 100527, 13, 5, 2], [0, 62, 82641, 289, 40908, 4, 28484, 678, 54529, 34695, 45, 289, 4, 98911, 7, 120709, 111, 70, 1257, 277, 214, 54452, 111, 44, 30404, 1294, 59566, 136, 70, 81206, 111, 70, 133720, 1242, 2, 2, 1734, 89971, 5908, 2045, 102449, 7, 8, 71200, 7, 125235, 733, 90, 8, 94, 30404, 1294, 59566, 82, 96, 25, 143045, 107, 115, 13000, 2746, 4084, 1251, 2], [0, 62, 332, 83, 123, 21896, 5, 2, 2, 8975, 45959, 3193, 8, 96, 25, 125415, 5, 2], [0, 62, 6, 85407, 81148, 83, 290, 14775, 98, 10, 47593, 5, 2, 2, 992, 118305, 6, 85407, 2219, 13, 1609, 51, 113444, 1507, 5, 2

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [115]:
encoded_train_dataset = train_dataset.map(preprocess_function, batched=True)
encoded_val_dataset = val_dataset.map(preprocess_function, batched=True)
encoded_test_dataset = test_dataset.map(preprocess_function, batched=True)

# print(encoded_train_dataset, encoded_val_dataset, encoded_test_dataset  )

Map:   0%|          | 0/1093 [00:00<?, ? examples/s]

Map:   0%|          | 0/137 [00:00<?, ? examples/s]

Map:   0%|          | 0/137 [00:00<?, ? examples/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about sentence classification, we use the `AutoModelForSequenceClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which is always 2, except for STS-B which is a regression problem and MNLI where we have 3 labels):

In [120]:
from transformers import RobertaForSequenceClassification, TrainingArguments, Trainer

num_labels = 1
model = RobertaForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using a model of type xlm-roberta to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define two more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [121]:
metric_name = "pearsonr"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
    logging_steps=10
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay. Since the best model might not be the one at the end of training, we ask the `Trainer` to load the best model it saved (according to `metric_name`) at the end of training.

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-mrpc"` or `"huggingface/bert-finetuned-mrpc"`).

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, the only preprocessing we have to do is to take the argmax of our predicted logits (our just squeeze the last axis in the case of STS-B):

In [122]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [123]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

You might wonder why we pass along the `tokenizer` when we already preprocessed our data. This is because we will use it once last time to make all the samples we gather the same length by applying padding, which requires knowing the model's preferences regarding padding (to the left or right? with which token?). The `tokenizer` has a pad method that will do all of this right for us, and the `Trainer` will use it. You can customize this part by defining and passing your own `data_collator` which will receive the samples like the dictionaries seen above and will need to return a dictionary of tensors.

We can now finetune our model by just calling the `train` method:

In [124]:
trainer.train()

Epoch,Training Loss,Validation Loss,Pearsonr
1,0.154300,0.120684,0.433619
2,0.158300,0.118905,0.451716
3,0.137100,0.125684,0.483404
4,0.108800,0.119753,0.490610
5,0.115300,0.104846,0.512706
6,0.069100,0.105471,0.593049
7,0.064300,0.090763,0.609191
8,0.058200,0.107179,0.592081
9,0.044400,0.107632,0.591130
10,0.043600,0.112350,0.590261


TrainOutput(global_step=690, training_loss=0.0958620814309604, metrics={'train_runtime': 590.7263, 'train_samples_per_second': 18.503, 'train_steps_per_second': 1.168, 'total_flos': 417525232149174.0, 'train_loss': 0.0958620814309604, 'epoch': 10.0})

We can check with the `evaluate` method that our `Trainer` did reload the best model properly (if it was not the last one):

In [101]:
trainer.evaluate()

{'eval_loss': 0.09831002354621887,
 'eval_pearsonr': 0.6358619528493408,
 'eval_runtime': 0.6236,
 'eval_samples_per_second': 219.699,
 'eval_steps_per_second': 14.433,
 'epoch': 10.0}

To see how your model fared you can compare it to the [GLUE Benchmark leaderboard](https://gluebenchmark.com/leaderboard).

You can now upload the result of the training to the Hub, just execute this instruction:

In [102]:
trainer.push_to_hub()

events.out.tfevents.1714721674.2b9cbacbba74.179.4:   0%|          | 0.00/23.0k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1714722212.2b9cbacbba74.179.5:   0%|          | 0.00/411 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/aabid123/xlm-roberta-base-finetuned-translation/commit/419c9519a4dcd483b0daa0c23e6e4d40b1f3eeac', commit_message='End of training', commit_description='', oid='419c9519a4dcd483b0daa0c23e6e4d40b1f3eeac', pr_url=None, pr_revision=None, pr_num=None)

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("sgugger/my-awesome-model")
```

## Hyperparameter search

The `Trainer` supports hyperparameter search using [optuna](https://optuna.org/) or [Ray Tune](https://docs.ray.io/en/latest/tune/). For this last section you will need either of those libraries installed, just uncomment the line you want on the next cell and run it.

In [103]:
! pip install optuna
! pip install ray[tune]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 15.7 MB/s eta 0:00:00


During hyperparameter search, the `Trainer` will run several trainings, so it needs to have the model defined via a function (so it can be reinitialized at each new run) instead of just having it passed. We jsut use the same function as before:

In [104]:
def model_init():
    return RobertaForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

And we can instantiate our `Trainer` like before:

In [105]:
trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

NameError: name 'encoded_dataset' is not defined

The method we call this time is `hyperparameter_search`. Note that it can take a long time to run on the full dataset for some of the tasks. You can try to find some good hyperparameter on a portion of the training dataset by replacing the `train_dataset` line above by:
```python
train_dataset = encoded_dataset["train"].shard(index=1, num_shards=10)
```
for 1/10th of the dataset. Then you can run a full training on the best hyperparameters picked by the search.

In [ ]:
best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize")

The `hyperparameter_search` method returns a `BestRun` objects, which contains the value of the objective maximized (by default the sum of all metrics) and the hyperparameters it used for that run.

In [ ]:
best_run

You can customize the objective to maximize by passing along a `compute_objective` function to the `hyperparameter_search` method, and you can customize the search space by passing a `hp_space` argument to `hyperparameter_search`. See this [forum post](https://discuss.huggingface.co/t/using-hyperparameter-search-in-trainer/785/10) for some examples.

To reproduce the best training, just set the hyperparameters in your `TrainingArgument` before creating a `Trainer`:

In [ ]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()